In [1]:
import numpy as np
import pandas as pd

In [2]:
from lightautoml_gpu.reader.gpu.cudf_reader import CudfReader
from lightautoml_gpu.reader.base import PandasToPandasReader

from lightautoml_gpu.transformers.base import SequentialTransformer

from lightautoml_gpu.pipelines.utils import get_columns_by_role

from lightautoml_gpu.transformers.gpu import numeric_gpu, categorical_gpu, datetime_gpu
from lightautoml_gpu.transformers import numeric, categorical, datetime

from lightautoml_gpu.tasks import Task
from lightautoml_gpu.automl.presets.tabular_presets import TabularAutoML, TabularUtilizedAutoML

import pandas as pd
import time
from sklearn.model_selection import train_test_split

from dask.distributed import Client
from dask_cuda import LocalCUDACluster
import cudf

In [3]:
features = pd.read_csv('../../data/small_new/multilabel/train Data.csv')
labels = pd.read_csv('../../data/small_new/multilabel/train labels.csv')
labels.drop(columns='id', inplace=True)
data = pd.concat([features, labels], axis=1)
data = data[['n_0000','n_0001','n_0002','n_0003',
             'n_0004','n_0005','n_0006','n_0007',
             'service_a', 'service_b', 'service_c', 
             'service_d', 'service_e', 'service_f',
             'service_g', 'service_h', 'service_i',
             'service_j', 'service_k', 'service_l',
             'service_m', 'service_n']]

tr_data, te_data = train_test_split(
    data, 
    test_size=0.2,  
    random_state=42
)

roles = {
    "target": {'service_a', 'service_b', 'service_c',
               'service_d', 'service_e', 'service_f',
               'service_g', 'service_h', 'service_i',
               'service_j', 'service_k', 'service_l',
               'service_m', 'service_n'},
    "drop" : ['id']
}

/tmp/ipykernel_8140/705777730.py:1: DtypeWarning: Columns (329,331,333,336,338,344,345,346,348,354,355,356,357,358,361,362,364,367,372,377,380,383,385,387,390,392,399,400,406,408,409,413,416,418,419,431,433,437,438,442,448,449,450,453,457,464,473,478,479,481,483,485,486,489,492,495,496,497,498,499,500,503,507,508,510,511,514,515,517,519,520,521,522,523,524,526,527,530,534,537,538,539,541,544,547,548,549,550,551,557,558,560,564,569,572,573,576,577,579,583,596,597,598,600,601,602,604,605,606,608,609,613,619,620,625,627,628,629,631,632,633,636,641,642,643,645,646,647,648,651,655,661,662,665,668,675,676,679,682,685,687,689,690,691,694,698,701,702,703,706,711,712,713,719,720,721,733,735,737,738,742,746,747,748,749,752,754,755,760,764,768,770,781,782,789,797,807,812,814,817,818,822,823,824,825,832,840,843,844,845,850,853,857,858,861,867,868,873,874,876,877,879,880,881,883,886,890,893,897,899,900,901,902,904,905,908,909,910,912,913,914,915,916,922,923,931,933,935,937,939,942,943,946,951,955,9

In [4]:
adv_roles = True

## Imports (for potential use)

In [5]:
# Imports from our package
from lightautoml_gpu.automl.base import AutoML

from lightautoml_gpu.automl.presets.gpu.tabular_gpu_presets import TabularAutoMLGPU, TabularUtilizedAutoMLGPU
from lightautoml_gpu.tasks import Task

from lightautoml_gpu.pipelines.features.gpu.lgb_pipeline_gpu import LGBSimpleFeaturesGPU, LGBAdvancedPipelineGPU
from lightautoml_gpu.pipelines.features.gpu.linear_pipeline_gpu import LinearFeaturesGPU

from lightautoml_gpu.pipelines.features.lgb_pipeline import LGBSimpleFeatures, LGBAdvancedPipeline
from lightautoml_gpu.pipelines.features.linear_pipeline import LinearFeatures


from lightautoml_gpu.ml_algo.gpu.boost_cb_gpu import BoostCBGPU
from lightautoml_gpu.ml_algo.gpu.boost_xgb_gpu import BoostXGB
from lightautoml_gpu.ml_algo.gpu.linear_gpu import LinearLBFGSGPU

from lightautoml_gpu.ml_algo.boost_cb import BoostCB
from lightautoml_gpu.ml_algo.linear_sklearn import LinearLBFGS


from lightautoml_gpu.pipelines.ml.base import MLPipeline
from lightautoml_gpu.pipelines.selection.importance_based import ModelBasedImportanceEstimator, ImportanceCutoffSelector

## TabularAutoML

In [6]:
task = Task('multilabel')

multilabel isn`t supported in lgb


In [7]:
automl = TabularAutoML(
    task = task, 
    timeout = 3600,
    cpu_limit = 4,
    reader_params = {'n_jobs': 4, 'cv': 3, 'random_state': 42},
    general_params = {'use_algos': [['linear_l2', 'cb']]}
)

In [8]:
oof_pred = automl.fit_predict(data, roles = roles, verbose = 2)

[22:14:24] Stdout logging level is INFO2.
[22:14:24] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[22:14:24] Task: multilabel

[22:14:24] Start automl preset with listed constraints:
[22:14:24] - time: 3600.00 seconds
[22:14:24] - CPU: 4 cores
[22:14:24] - memory: 16 GB

[22:14:24] Train data shape: (14644, 22)



/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml_gpu/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml_gpu/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0


[22:14:28] Feats was rejected during automatic roles guess: []
[22:14:28] Layer 1 train process start. Time left 3596.57 secs
[22:14:28] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[22:14:28] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml_gpu/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml_gpu/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml_gpu/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/lightautoml_gpu/reader/guess_roles.py:56: RuntimeWarning: invalid value encountered in float_scalars
  gini_sum = a_c.sum() / a_s.sum() - (n + 1) / 2.0


[22:14:31] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====
[22:14:34] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_0_LinearL2 =====
[22:14:36] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = -7.35028162615705
[22:14:36] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[22:14:36] Time left 3588.06 secs



...

[22:14:52] ===== Start working with fold 1 for Lvl_0_Pipe_1_Mod_0_CatBoost =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


[22:15:00] ===== Start working with fold 2 for Lvl_0_Pipe_1_Mod_0_CatBoost =====


/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,
/home/rishat/miniconda3/envs/rapids-22.10/lib/python3.9/site-packages/catboost/core.py:1419: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  self._init_pool(data, label, cat_features, text_features, embedding_features, embedding_features_data, pairs, weight,


[22:15:08] Fitting Lvl_0_Pipe_1_Mod_0_CatBoost finished. score = -7.319639591038699
[22:15:08] Lvl_0_Pipe_1_Mod_0_CatBoost fitting and predicting completed
[22:15:08] Time left 3556.05 secs

[22:15:08] Layer 1 training completed.

[22:15:08] Blending: optimization starts with equal weights and score -7.314009267074656
[22:15:08] Blending: iteration 0: score = -7.313950612738106, weights = [0.5435342 0.4564658]
[22:15:09] Blending: iteration 1: score = -7.313950612738106, weights = [0.5435342 0.4564658]
[22:15:09] Blending: no score update. Terminated

[22:15:09] Automl preset training completed in 44.32 seconds

[22:15:09] Model description:
Final prediction for new objects (level 0) = 
	 0.54353 * (3 averaged models Lvl_0_Pipe_0_Mod_0_LinearL2) +
	 0.45647 * (3 averaged models Lvl_0_Pipe_1_Mod_0_CatBoost) 



In [6]:
task = Task('multilabel', device='gpu')

multilabel isn`t supported in lgb


In [7]:
automl_gpu = TabularAutoMLGPU(
    task = task, 
    timeout = 3600,
    cpu_limit = 1,
    reader_params = {'n_jobs': 1, 'cv': 3, 'random_state': 42},
    general_params = {'use_algos': [['linear_l2', 'pb']]}
)

In [8]:
oof_pred_gpu = automl_gpu.fit_predict(data, roles = roles, verbose = 2)

[22:37:34] Stdout logging level is INFO2.
[22:37:34] Copying TaskTimer may affect the parent PipelineTimer, so copy will create new unlimited TaskTimer
[22:37:34] Task: multilabel

[22:37:34] Start automl preset with listed constraints:
[22:37:34] - time: 3600.00 seconds
[22:37:34] - CPU: 1 cores
[22:37:34] - memory: 16 GB

[22:37:34] Train data shape: (14644, 22)
[22:37:34] Feats was rejected during automatic roles guess: []
[22:37:34] Layer 1 train process start. Time left 3599.31 secs
[22:37:35] Start fitting Lvl_0_Pipe_0_Mod_0_LinearL2 ...
[22:37:35] ===== Start working with fold 0 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[22:37:41] ===== Start working with fold 1 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[22:37:47] ===== Start working with fold 2 for Lvl_0_Pipe_0_Mod_0_LinearL2 (orig) =====
[22:37:53] Fitting Lvl_0_Pipe_0_Mod_0_LinearL2 finished. score = -7.377902406982404
[22:37:53] Lvl_0_Pipe_0_Mod_0_LinearL2 fitting and predicting completed
[22:37:53] Time left 3580.44 s

In [9]:
gpu_inf = automl_gpu.predict(data)

In [10]:
automl_gpu.to_cpu()

multilabel isn`t supported in lgb
[22:38:24] CatBoost uses as obj. MultiCrossEntropy.
multilabel isn`t supported in lgb
[22:38:24] CatBoost uses as obj. MultiCrossEntropy.
multilabel isn`t supported in lgb
[22:38:24] CatBoost uses as obj. MultiCrossEntropy.
multilabel isn`t supported in lgb
[22:38:25] CatBoost uses as obj. MultiCrossEntropy.


  0%|          | 0/137 [00:00<?, ?it/s]

  0%|          | 0/134 [00:00<?, ?it/s]

  0%|          | 0/125 [00:00<?, ?it/s]

multilabel isn`t supported in lgb
[22:38:26] CatBoost uses as obj. MultiCrossEntropy.


In [11]:
cpu_inf = automl_gpu.predict(data)

In [1]:
from joblib import dump, load
import time
pickle_file = './old_gpu.joblib'

In [14]:
start = time.time()
with open(pickle_file, 'wb') as f:
    dump(automl_gpu, f)
raw_dump_duration = time.time() - start
print("Raw dump duration: %0.3fs" % raw_dump_duration)

Raw dump duration: 0.162s


In [2]:

start = time.time()
with open(pickle_file, 'rb') as f:
    model_inf = load(f)
raw_load_duration = time.time() - start
print("Raw load duration: %0.3fs" % raw_load_duration)


Raw load duration: 2.589s


In [12]:
model_inf.levels[0][1].pre_selection.

In [16]:
disk_pred = model_inf.predict(data)

In [17]:
disk_pred.data.T

array([[0.00155754, 0.0025776 , 0.00147561, ..., 0.00532673, 0.00168314,
        0.00187416],
       [0.15798661, 0.17665237, 0.13783291, ..., 0.32286385, 0.10485923,
        0.15111391],
       [0.00420095, 0.01183663, 0.00461977, ..., 0.00896302, 0.00529343,
        0.0043902 ],
       ...,
       [0.09696509, 0.13706353, 0.0923971 , ..., 0.26710537, 0.08439717,
        0.11637649],
       [0.20799798, 0.25122336, 0.18583645, ..., 0.43225628, 0.15812257,
        0.22918661],
       [0.6083876 , 0.5579189 , 0.6050685 , ..., 0.34639215, 0.6165759 ,
        0.5700073 ]], dtype=float32)

In [18]:
disk_pred.data.T

array([[0.00155754, 0.0025776 , 0.00147561, ..., 0.00532673, 0.00168314,
        0.00187416],
       [0.15798661, 0.17665237, 0.13783291, ..., 0.32286385, 0.10485923,
        0.15111391],
       [0.00420095, 0.01183663, 0.00461977, ..., 0.00896302, 0.00529343,
        0.0043902 ],
       ...,
       [0.09696509, 0.13706353, 0.0923971 , ..., 0.26710537, 0.08439717,
        0.11637649],
       [0.20799798, 0.25122336, 0.18583645, ..., 0.43225628, 0.15812257,
        0.22918661],
       [0.6083876 , 0.5579189 , 0.6050685 , ..., 0.34639215, 0.6165759 ,
        0.5700073 ]], dtype=float32)

In [19]:
print(cpu_inf.data.T)
print()
print(gpu_inf.data.T)
print()
print(oof_pred_gpu.data.T)
print()
print(oof_pred.data.T)
print()
print(disk_pred.data.T)

[[0.0017595  0.00286257 0.00165218 ... 0.00575643 0.00188154 0.00198671]
 [0.17117482 0.18910322 0.15029469 ... 0.34120476 0.11212785 0.15767835]
 [0.00441826 0.01217545 0.00491824 ... 0.00829808 0.00542473 0.00417888]
 ...
 [0.10361604 0.14414078 0.0989444  ... 0.27280742 0.09031855 0.11665703]
 [0.21969722 0.2604978  0.19769078 ... 0.4448157  0.16723959 0.23986948]
 [0.60571265 0.5560098  0.6020238  ... 0.35372484 0.6146251  0.5679983 ]]

[[0.00155754 0.0025776  0.00147561 ... 0.00532673 0.00168314 0.00187416]
 [0.15798661 0.17665237 0.13783291 ... 0.32286385 0.10485927 0.15111391]
 [0.00420095 0.01183663 0.00461977 ... 0.00896303 0.00529344 0.0043902 ]
 ...
 [0.09696507 0.13706358 0.09239709 ... 0.26710537 0.08439717 0.11637649]
 [0.20799798 0.25122336 0.18583646 ... 0.43225625 0.15812257 0.22918656]
 [0.60838753 0.55791897 0.6050685  ... 0.3463921  0.6165759  0.5700073 ]]

[[0.00135836 0.00258094 0.00147505 ... 0.0059649  0.00189636 0.00173164]
 [0.14873028 0.1667052  0.14298013 ..

NameError: name 'oof_pred' is not defined

In [ ]:
cluster = LocalCUDACluster(rmm_managed_memory=True, CUDA_VISIBLE_DEVICES="0",
                               protocol="ucx", enable_nvlink=True,
                               memory_limit="8GB")
print("dashboard:", cluster.dashboard_link)
client = Client(cluster)
client.run(cudf.set_allocator, "managed")

In [ ]:
task = Task('multilabel', device='mgpu')

automl_mgpu = TabularAutoMLGPU(
    task = task, 
    timeout = 3600,
    cpu_limit = 1,
    reader_params = {'n_jobs': 1, 'cv': 3, 'random_state': 42, 'npartitions': 2},
    general_params = {'use_algos': [['xgb', 'linear_l2']]},
    client = client
)

oof_pred_mgpu = automl_mgpu.fit_predict(data, roles = roles, verbose = 2)

In [ ]:
automl_mgpu.to_cpu()

In [ ]:
mcpu_inf = automl_mgpu.predict(data)

In [ ]:
print(cpu_inf.data.T)
print()
print(gpu_inf.data.T)
print()
print(oof_pred_gpu.data.T)
print()
print(oof_pred.data.T)
print()
print(oof_pred_mgpu.data.T)
print()
print(mcpu_inf.data.T)

In [ ]:
from joblib import dump, load
import time

pickle_file = './mgpu.joblib'

start = time.time()
with open(pickle_file, 'wb') as f:
    dump(automl_mgpu, f)
raw_dump_duration = time.time() - start
print("Raw dump duration: %0.3fs" % raw_dump_duration)